# Migrate SQL Server Database to Azure SQL DB

## Overview

Use this notebook when attempting to migrate a SQL Server database to Azure SQL Database. The process is different than using an Azure SQL Virtual Machine or Azure SQL Managed Instance. Refer to this [Microsoft docs](https://docs.microsoft.com/en-us/azure/dms/howto-sql-server-to-azure-sql-powershell) article for the method described.

## Variables

Use the table below as reference when configuring this notebook for execution.

| Line | Variable | Description | Example |
| --- | --- | --- | --- |
| 1 | Subscription | Azure subscription name or ID | guid |
| 2 | dmsName | Name of the Data Migration Service | MyDms |
| 3 | dmsResourceGroup | Exact name of resource group to target or create | _MyAz\_RG_ |
| 4 | loc | Specifies the location of the service | EastUS2 |
| 5 | vNetName | Specify the name of the vnet Azure resource to use | vnet1 |
| 6 | Subnet | Name of the Azure subnet resource to use | subnet1 |
| 7-8 | dmsSku | This parameter corresponds to DMS Sku name. The currently supported sku is listed. | GeneralPurpose\_4vCores |
| 9 | TargetSqlServer | Target SQL server to migrate to | "sqllazuretarget.database.windows.net" |
| 10 | SourceDb | Name of database to migrate from, the _source_ | AdventureWorks2016 |
| 11 | TargetDb | Name of database to migrate to, in case it differs from the source, the _target_ | AdventureWorks2016 |

Lastly, specify the table names explicitly as a PowerShell list of names:

> `$Tables = @(`<span style="color: rgb(215, 186, 125); font-family: Menlo, Monaco, Consolas, &quot;Droid Sans Mono&quot;, &quot;Courier New&quot;, monospace, &quot;Droid Sans Fallback&quot;; font-size: 12px;">"HumanResources.Department",&nbsp;</span> <span style="color: rgb(215, 186, 125); font-family: Menlo, Monaco, Consolas, &quot;Droid Sans Mono&quot;, &quot;Courier New&quot;, monospace, &quot;Droid Sans Fallback&quot;; font-size: 12px;">"HumanResources.Employee",&nbsp;</span> <span style="color: rgb(215, 186, 125); font-family: Menlo, Monaco, Consolas, &quot;Droid Sans Mono&quot;, &quot;Courier New&quot;, monospace, &quot;Droid Sans Fallback&quot;; font-size: 12px;">"HumanResources.EmployeeDepartmentHistory")</span>

In [ ]:
$Subscription = ""
$dmsName = ""
$dmsResourceGroup = ""
$loc = ""
$vNetName = ""
$Subnet = Subnet1
$dmsSku = GeneralPurpose_4vCores 
$TargetSqlServer = ""
$SourceDb = ""
$TargetDb = ""

# Define a list of tables to migrate
$Tables = @()
<# :::For example::: 
$Tables = @("HumanResources.Department", `
            "HumanResources.Employee", `
            "HumanResources.EmployeeDepartmentHistory", `
            "HumanResources.EmployeePayHistory", `
            "HumanResources.JobCandidate", `
            "HumanResources.Shift")
#>

## Connections

In [ ]:
# Configure virtual network
$vNet = Get-AzVirtualNetwork -ResourceGroupName $dmsResourceGroup -Name $vNetName

$vSubNet = Get-AzVirtualNetworkSubnetConfig -VirtualNetwork $vNet -Name $Subnet

# Create a Database Connection Info object for the source connections
$sourceConnInfo = New-AzDmsConnInfo -ServerType SQL `
    -DataSource $SourceSqlServer `
    -AuthType SqlAuthentication `
    -TrustServerCertificate:$true

# target connection
$targetConnInfo = New-AzDmsConnInfo -ServerType SQL `
    -DataSource $TargetSqlServer `
    -AuthType SqlAuthentication `
    -TrustServerCertificate:$false

## Select Data by Defining Database and Table Mapping

In [ ]:
# Create a list of databases to migrate
$dbInfo1 = New-AzDataMigrationDatabaseInfo -SourceDatabaseName $SourceDb
$dbList = @($dbInfo1)

# Create a table map 
$tableMap = New-Object 'system.collections.generic.dictionary[string,string]'
foreach ($table in $Tables)
{
    $tableMap.Add($table,$table)
}

# Select the data
$selectedDbs = New-AzDmsSelectedDB -MigrateSqlServerSqlDb -Name $SourceDb `
    -TargetDatabaseName $TargetDb `
    -TableMap $tableMap

## Create Service Project

In [ ]:
$service = New-AzDms -ResourceGroupName $dmsResourceGroup `
    -ServiceName $dmsName `
    -Location $loc `
    -Sku $dmsSku `
    -VirtualSubnetId $vSubNet.Id

# Create a project object
$project = New-AzDataMigrationProject -ResourceGroupName $dmsResourceGroup `
    -ServiceName $service.Name `
    -ProjectName $dmsNameProject `
    -Location $loc `
    -SourceType SQL `
    -TargetType SQLDB `
    -SourceConnection $sourceConnInfo `
    -TargetConnection $targetConnInfo `
    -DatabaseInfo $dbList

## Credentials for Source and Target

In [ ]:
$secpasswd = ConvertTo-SecureString -String $sourcePassword -AsPlainText -Force
$sourceCred = New-Object System.Management.Automation.PSCredential ($sourceUserName, $secpasswd)
$secpasswd = ConvertTo-SecureString -String $targetPassword -AsPlainText -Force
$targetCred = New-Object System.Management.Automation.PSCredential ($targetUserName, $secpasswd)

## Create the Migration Task
Uncomment lines to perform validation checks during the task.

In [ ]:
$migTask = New-AzDataMigrationTask -TaskType MigrateSqlServerSqlDb `
    -ResourceGroupName $dmsResourceGroup `
    -ServiceName $service.Name `
    -ProjectName $project.Name `
    -TaskName $dmsNameTask `
    -SourceConnection $sourceConnInfo `
    -SourceCred $sourceCred `
    -TargetConnection $targetConnInfo `
    -TargetCred $targetCred `
    -SelectedDatabase  $selectedDbs `
#    -SchemaValidation `
#    -DataIntegrityValidation `
#    -QueryAnalysisValidation `

## Monitor the Migration Task

In [ ]:
# Monitor the migration
if (($migTask.ProjectTask.Properties.State -eq "Running") -or ($migTask.ProjectTask.Properties.State -eq "Queued"))
{
  write-host "migration task running"
}
else 
{
    Write-Host "migration task is $migTask.ProjectTask.Properties.State"
}

## Deleting the DMS Instance
This should remove the DMS instance from the Resource Group. Uncomment to perform the task. 

In [ ]:
#Remove-AzDms -ResourceGroupName $dmsResourceGroup -ServiceName $dmsName